In [35]:
import gymnasium as gym
import gymnasium_env
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback
import os 
import torch
import tensorboard

In [16]:
from gymnasium.envs.registration import register

try:
    register(
        id="gymnasium_env/GridWorld-v0",
        entry_point="gymnasium_env.envs:GridWorldEnv",
    )
    print("Environment 'gymnasium_env/GridWorld-v0' registered successfully.")
except gym.error.AlreadyRegistered:
    print("Environment 'gymnasium_env/GridWorld-v0' already registered.")

Environment 'gymnasium_env/GridWorld-v0' registered successfully.


In [ ]:
train_env = make_vec_env(
    "gymnasium_env/GridWorld-v0",
    n_envs=4,
    seed=0, 
    env_kwargs={"render_mode": None}
    )

In [28]:
eval_env = gym.make("gymnasium_env/GridWorld-v0", render_mode="human")

In [20]:
log_dir = "./ppo_gridworld_logs_eval/"
os.makedirs(log_dir, exist_ok=True)

In [22]:
TOTAL_TIMESTEPS = 200000

In [27]:
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=log_dir,
    log_path=log_dir,
    eval_freq=TOTAL_TIMESTEPS // 10, 
    deterministic=True,
    render=False, 
    n_eval_episodes=5 
)

In [29]:
observation, info = eval_env.reset()
img = eval_env.render()

In [12]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.current_device())
    print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA GeForce RTX 3060 Laptop GPU


In [13]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)        

cuda


In [30]:
model = PPO(
    "MultiInputPolicy", 
    train_env, verbose=1, device=device,
    tensorboard_log="./tensorboard_logs/")


Using cuda device


In [36]:
model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=eval_callback)
model.save("ppo_gridworld")

ImportError: Trying to log data to tensorboard but tensorboard is not installed.

In [16]:
env.close()